In [1]:
from keras.applications import inception_v3
from keras import backend as K

Using TensorFlow backend.


In [2]:
K.set_learning_phase(0)
model=inception_v3.InceptionV3(weights='imagenet',include_top=False)

87916544/87910968 [==============================] - 705s 8us/step


In [3]:
layer_contributions={'mixed2':0.2,'mixed3':3.,'mixed4':2.,'mixed5':1.5,}

In [6]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss = loss+coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

In [7]:
dream=model.input
grads=K.gradients(loss,dream)[0]
grads/=K.maximum(K.mean(K.abs(grads)),1e-7)
outputs=[loss,grads]
fetch_loss_and_grads=K.function([dream],outputs)
def eval_loss_and_grads(x):
  outs=fetch_loss_and_grads([x])
  loss_value=outs[0]
  grads_values=outs[1]
  return loss_value,grads_values
def gradient_ascent(x,iterations,step,max_loss=None):
  for i in range(iterations):
    loss_value,grads_values=eval_loss_and_grads(x)
    if max_loss is not None and loss_value>max_loss:
      break;
    print(i,'loss',loss_value)
    x+=step*grads_values
  return x

In [8]:
import scipy
from keras.preprocessing import image

In [20]:
def resize_img(img,size):
  img=np.copy(img)
  factors=(1,float(size[0])/img.shape[1],float(size[1])/img.shape[2],1)
  return scipy.ndimage.zoom(img,factors,order=1)

def save_imgs(img, fname):
    pil_img = deprocess_image(np.copy(img))
    image.save_img(fname, pil_img)
def preprocess_image(image_path):
  img=image.load_img(image_path)
  img=image.img_to_array(img)
  img=np.expand_dims(img,axis=0)
  img=inception_v3.preprocess_input(img)
  return img
def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [22]:
import numpy as np
step=0.01
num_octave=3
octave_scale=1.4
iterations=20
max_loss=10.

base_image_path=r'D:\keras\original_photo_deep_dream.jpg'
img=preprocess_image(base_image_path)
original_shape=img.shape[1:3]
sucessive_shapes=[original_shape]
for i in range(1,num_octave):
  shape=tuple([int(dim/(octave_scale**i)) for dim in original_shape])
  sucessive_shapes.append(shape)
sucessive_shapes=sucessive_shapes[::-1]
original_img=np.copy(img)
shrunk_original_img=resize_img(img,sucessive_shapes[0])
for shape in sucessive_shapes:
  print("size",shape)
  img=resize_img(img,shape)
  img=gradient_ascent(img,iterations=iterations,step=step,max_loss=max_loss)
  upscaled_shrunk_original_img=resize_img(shrunk_original_img,shape)
  same_size_original=resize_img(original_img,shape)
  lost_detail=same_size_original-upscaled_shrunk_original_img
  img+=lost_detail
  shrunk_original_img=resize_img(original_img,shape)
  save_imgs(img,fname='dream_at_scale'+str(shape)+'.png')

save_imgs(img,fname=r'D:\keras\final_dream.png')

size (178, 178)
0 loss 0.65961766
1 loss 1.0167829
2 loss 1.4680848
3 loss 2.0043151
4 loss 2.608424
5 loss 3.0937617
6 loss 3.5753698
7 loss 4.057526
8 loss 4.4507847
9 loss 4.888896
10 loss 5.337751
11 loss 5.773019
12 loss 6.177859
13 loss 6.5435257
14 loss 6.962656
15 loss 7.3771544
16 loss 7.7456737
17 loss 8.149124
18 loss 8.544074
19 loss 8.879556
size (250, 250)
0 loss 2.1796741
1 loss 3.5691357
2 loss 4.625678
3 loss 5.541784
4 loss 6.442739
5 loss 7.228117
6 loss 7.912153
7 loss 8.624628
8 loss 9.325433
9 loss 9.958045
size (350, 350)
0 loss 2.3517466
1 loss 3.4891381
2 loss 4.574066
3 loss 5.5842443
4 loss 6.603441
5 loss 7.6351595
6 loss 8.924164
